<a href="https://colab.research.google.com/github/hunar0710/Real-time-data-ingestion-Final-Project/blob/main/Real_time_data_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required Python packages: PySpark for Spark processing, delta-spark for Delta Lake support
!pip install pyspark==3.4.1 delta-spark faker schedule

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of delta-spark to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of delta-spark to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 95.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285391 sha256=c6b330d8ad864e942b5c0238273f0890c3b714adfaf3e5afe1e59b5a188fcf59
  Stored in directory: /root/.cache/pip/wheels/e9/b4/d8/38accc42606f6675165423e9f0236f8e825f6b6b6048d6743e
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
ERROR: pip's dependency resolver does not current

In [ ]:
# Step 2: Create and configure the Spark session with Delta Lake support and set the timezone to Asia/Kolkata
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
builder = SparkSession.builder \
    .appName("DeltaTableColab") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.warehouse.dir", "/content/delta-warehouse")
spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.conf.set("spark.sql.session.timeZone", "Asia/Kolkata")

In [ ]:
# Step 3: Generate fake data using the Faker library
from faker import Faker
import pandas as pd
fake = Faker()
def generate_fake_data(n=5):
    data = [{
        "Name": fake.name(),
        "Address": fake.address(),
        "Email": fake.email()
    } for _ in range(n)]
    return pd.DataFrame(data)

In [ ]:
# Step 4: Append the generated fake data to a Delta Lake table
from pyspark.sql import DataFrame
def append_to_delta(df: pd.DataFrame, path: str):
    spark_df = spark.createDataFrame(df)
    spark_df.write.format("delta").mode("append").save(path)
    print("Appended data:")
    spark.read.format("delta").load(path).show()

In [ ]:
# Step 5: Generate an HTML summary of the DataFrame
def generate_html_summary(df: pd.DataFrame):
    return df.to_html(index=False)

In [ ]:
# Step 6: Track and display the version history of the Delta table
from delta.tables import DeltaTable
def track_versions(path: str):
    delta_table = DeltaTable.forPath(spark, path)
    print(" Delta Table Version History:")
    delta_table.history().show(truncate=False)

In [ ]:
# Step 7: Define and run the complete data ingestion pipeline
def run_pipeline():
    delta_path = "/content/delta-table"
    df = generate_fake_data()
    append_to_delta(df, delta_path)
    html_summary = generate_html_summary(df)
    from IPython.display import display, HTML
    display(HTML(html_summary))
    track_versions(delta_path)

In [ ]:
# Step 8: Schedule the ingestion pipeline to run every 5 minutes
import schedule
import time
schedule.every(5).minutes.do(run_pipeline)
print(" Starting the ingestion pipeline (runs every 5 mins)...")
while True:
    schedule.run_pending()
    time.sleep(1)

 Starting the ingestion pipeline (runs every 5 mins)...
Appended data:
+---------------+--------------------+--------------------+
|           Name|             Address|               Email|
+---------------+--------------------+--------------------+
|   Steven Allen|6842 Christy Port...|   cyang@example.net|
| Katie Matthews|8861 Pham Skyway\...| karla52@example.com|
|James Patterson|2624 Carter Gatew...|  xbaker@example.net|
|      Gail Hays|Unit 8568 Box 484...|stephanie33@examp...|
| Yesenia Osborn|9097 Johnson Rout...|tsingleton@exampl...|
+---------------+--------------------+--------------------+



Name,Address,Email
Gail Hays,Unit 8568 Box 4846\nDPO AP 35916,stephanie33@example.net
Yesenia Osborn,"9097 Johnson Route Suite 366\nWest Isaiah, NH 10337",tsingleton@example.com
Steven Allen,"6842 Christy Port\nHeatherberg, AK 27157",cyang@example.net
Katie Matthews,"8861 Pham Skyway\nLake Patrick, GU 81336",karla52@example.com
James Patterson,"2624 Carter Gateway Suite 618\nNew Jerry, MP 54352",xbaker@example.net


 Delta Table Version History:
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|0      |2025-07-18 16:50:37.223|null  |null    |WRITE    |{mode -> Append, partitionBy -> []}|null|null    |null     |null       |Serializable  |true         |

Name,Address,Email
Christian Lindsey,Unit 0888 Box 7027\nDPO AA 57970,joshua09@example.org
Paula Meyer,"7114 Andrew Fall\nDavidton, CT 74802",anthony08@example.org
Kathleen Conway,"07623 Susan Mission Apt. 616\nSouth John, VT 40848",lgonzalez@example.net
Paul Jones,"07908 Smith Street Suite 755\nKevinport, AR 97504",marcusfreeman@example.net
Sarah Johnson,"3577 Phillips Lights\nJohnsonborough, TX 53523",myersjessica@example.org


 Delta Table Version History:
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|1      |2025-07-18 16:55:51.942|null  |null    |WRITE    |{mode -> Append, partitionBy -> []}|null|null    |null     |0          |Serializable  |true         |

...

 Delta Table Version History:
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|8      |2025-07-18 17:31:38.84 |null  |null    |WRITE    |{mode -> Append, partitionBy -> []}|null|null    |null     |7          |Serializable  |true         |

Name,Address,Email
Nathaniel Welch,"3346 Debra Landing Apt. 517\nLake Jessicaburgh, AZ 09664",justinjones@example.com
Michael Griffin,Unit 7847 Box 6825\nDPO AA 68933,lauren88@example.com
Susan Ramirez,"861 Rice Fields\nBarkerport, MA 67143",murraykelly@example.com
Rose Stone,"026 Perez Shores\nChristinachester, AZ 02162",hgonzales@example.com
Seth Rivera,"49801 Matthews Ridges\nNorth Tyler, MP 63730",ddaugherty@example.net


 Delta Table Version History:
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|9      |2025-07-18 17:36:44.664|null  |null    |WRITE    |{mode -> Append, partitionBy -> []}|null|null    |null     |8          |Serializable  |true         |

Name,Address,Email
Kimberly Tucker,"595 Maynard Tunnel Apt. 841\nCrawfordburgh, CO 58748",phyllis17@example.net
Xavier Edwards,"86915 Kenneth Estate Apt. 485\nPort Stephen, ME 85624",xchen@example.net
Michael Thompson PhD,"3986 Shawn Cape Apt. 379\nWilsonmouth, RI 18481",robertmoore@example.net
Timothy Williams,"0149 Jeremiah Junction Apt. 453\nConnieshire, FL 66519",jessica60@example.net
Jennifer Fields,"53272 Mendoza Alley\nCharleschester, GA 47461",nlozano@example.net


 Delta Table Version History:
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|10     |2025-07-18 17:41:50.074|null  |null    |WRITE    |{mode -> Append, partitionBy -> []}|null|null    |null     |9          |Serializable  |true         |